# Homework 01

In [1]:
!python -V

Python 3.11.12


In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

### Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [2]:
# Download the data
january_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
february_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
# Number of columns for January data
print(f"There are {len(january_data.columns)} columns in January data.")

There are 20 columns in January data.


### Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [3]:
january_data['duration'] = january_data.tpep_dropoff_datetime - january_data.tpep_pickup_datetime
january_data.duration = january_data.duration.apply(lambda td: td.total_seconds() / 60)
january_duration = round(january_data.duration.std(), 2)
print(f"The standard deviation of the trips duration in January {january_duration}")

The standard deviation of the trips duration in January 42.59


### Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [5]:
january_data_clean = january_data[(january_data.duration >= 1) & (january_data.duration <= 60)]
records_after_drop = round(january_data_clean.shape[0] * 100 / january_data.shape[0])
print(f"Fraction of the records left after we dropped the outliers is {records_after_drop}")

Fraction of the records left after we dropped the outliers is 98


### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will
  label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [11]:
categorical = ['PULocationID', 'DOLocationID']
train_dicts = january_data_clean[categorical].to_dict(orient='records')

dv = DictVectorizer()
x_train = dv.fit_transform(train_dicts)
print(f"The dimensionality of the matrix is {x_train.shape[1]}")

The dimensionality of the matrix is 515


### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [33]:
y_train = january_data_clean['duration'].values
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)
y_predict = lin_reg.predict(x_train)
mse = mean_squared_error(y_train, y_predict)
rmse = round(np.sqrt(mse), 3)
print(f"RMSE on train is: {rmse}")

RMSE on train is: 7.649


### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [34]:
y_train = january_data_clean['duration'].values
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

y_pred_train = lin_reg.predict(x_train)
rmse_train = round(root_mean_squared_error(y_train, y_pred_train), 3)
print(f"RMSE on train is: {rmse_train}")

february_data['duration'] = february_data.tpep_dropoff_datetime - february_data.tpep_pickup_datetime
february_data['duration'] = february_data['duration'].apply(lambda td: td.total_seconds() / 60)

february_data_clean = february_data[(february_data.duration >= 1) & (february_data.duration <= 60)]

val_dicts = february_data_clean[categorical].astype(str).to_dict(orient='records')
x_val = dv.transform(val_dicts)
y_val = february_data_clean['duration'].values

y_pred_val = lin_reg.predict(x_val)
rmse_val = round(root_mean_squared_error(y_val, y_pred_val), 2)
print(f"RMSE on validation is: {rmse_val}")

RMSE on train is: 7.649
RMSE on validation is: 7.81
